In [33]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from hashlib import sha256
from itertools import combinations
from statsmodels.stats.multitest import multipletests
import scipy
from scipy.stats import skew
from distfit import distfit
from statsmodels.stats.power import tt_ind_solve_power
import ipywidgets as widgets
from IPython.display import display

# Data loading

### Metrics selection

In [10]:
data0 = pd.read_csv("rekko_split_v1_1_transitioned.csv")
var_names = ['test_user_id', 'tag', 'watched_time', 'total_revenue', 'selects_from_rails', 'twt_rails', 'twt_rekko_rails']
data0["watched_time"] = data0["watched_time"]/3600
data0["twt_rails"] = data0["twt_rails"]/3600
data0["twt_rekko_rails"] = data0["twt_rekko_rails"]/3600
data0["total_revenue"] = data0["tvod_revenue"] + data0["svod_revenue"]
data0["total_purchases"] = data0["tvod_purchases"] + data0["svod_purchases"]
data1 = data0[var_names]

## Splitting

In [ ]:
data1_test1 = data1.loc[data1['tag'] == 'test1']
data1_control = data1.loc[data1['tag'] == 'control']
data1.columns

### Removing very large outliers

In [12]:
exclude_cols = ['test_user_id', 'tag']
for col in data1_test1.columns:
    if col in exclude_cols:
        continue
    threshold1 = data1_test1[col].quantile(0.99999)
    above_threshold1 = data1_test1[data1_test1[col] <= threshold1]
    data1_test1 = above_threshold1.copy()

In [13]:
for col in data1_control.columns:
    if col in exclude_cols:
        continue
    threshold3 = data1_control[col].quantile(0.99999)
    above_threshold3 = data1_control[data1_control[col] <= threshold3]
    data1_control = above_threshold3.copy()

### Final dataset

In [14]:
data1 = pd.concat([data1_test1, data1_control], ignore_index=True)

In [ ]:
metric_names = ['watched_time', 'twt_rails', 'twt_rekko_rails', 'total_revenue', 'selects_from_rails']

stats2 = data1_control[metric_names].describe(percentiles=[.01,.05,.25,.50,.75, .95,.99])
#stats2 = data1_test1[metric_names].describe(percentiles=[.01,.05,.25,.50,.75, .95,.99])

skewness = data1_control[metric_names].apply(skew)

stats2.loc['skewness'] = skewness


num_zeros = (data1_control[metric_names] == 0).sum()

# Calculate the total number of rows
total_rows = data1_control[metric_names].shape[0]

# Calculate the percentage of zeros
percentage_zeros = (num_zeros / total_rows) * 100

stats2.loc['zeros'] = percentage_zeros
stats2.round(2)

# Metrics distribution


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Function to plot distribution for a given metric and data type
def plot_distribution(metric, data_type):
    plt.figure(figsize=(10, 6))
    plt.suptitle(f"Distribution of {metric} - {data_type}", y=1.05)
    #plt.subplots_adjust(top=0.85)

    data = None
    color = None

    if data_type == 'test1':
        data = data1_test1
        color = 'red'
    elif data_type == 'control':
        data = data1_control
        color = 'blue'
    else:
        print("Invalid data type. Please choose from 'test1', 'test2', or 'control'.")
        return
    
    #plt.subplot(1, 1, 1)
    positive_data = [x for x in data[metric] if x > 0]
    sns.histplot(positive_data,kde="True", label=data_type, color=color, bins = 100)
    plt.xlim()
    plt.xlabel("Values")
    plt.ylabel('Frequency')
    plt.show()

# Input the metric of your choice
metric = input("Enter the metric to display distribution: ")

plot_distribution(metric, 'test1')
plot_distribution(metric, 'control')

# Deltas distribution

In [28]:
# Function to calculate deltas for a specific metric
def calculate_delta(metric_name, test_data, control_data):
    test_data = test_data.sort_values(ascending=False).reset_index(drop=True)
    control_data = control_data.sort_values(ascending=False).reset_index(drop=True)

    min_len = min(len(test_data), len(control_data))
    delta = []
    for i in range(min_len):
        if test_data[i] == 0 and control_data[i] == 0:
            continue  # skip subtraction if both values are zero
        else:
            delta.append(test_data[i] - control_data[i])
    delta = pd.Series(delta).sort_values()
    return delta

In [ ]:
# Function to display delta for a specific metric
def display_delta(metric_name, delta):
    print("Delta for", metric_name, ":", delta)

# Function to display delta histogram for a specific metric
def display_delta_histogram(metric_name, delta):
    
        range_start, range_end = np.min(delta), np.max(delta)
        title = "Delta Distribution for " + metric_name

        sns.histplot(delta, stat='density', color='darkblue', kde="True",bins=100)
        plt.title(title)
        plt.xlabel("Delta Value")
        plt.ylabel("Density")
        plt.xlim(range_start, range_end)  # Set x-axis range
        plt.show()

# Input the metric name for which you want to display the delta
metric_name_input = input("Enter the metric name for which you want to display the delta and histogram: ")

# Check if the input metric name is valid
if metric_name_input in metric_names:
    test1_data = data1_test1[metric_name_input]
    control_data = data1_control[metric_name_input]
    

    # Calculate deltas
    delta_test1 = calculate_delta(metric_name_input, test1_data, control_data)

    # Display deltas for the chosen metric
    display_delta(metric_name_input, delta_test1)

    # Display delta histogram for the chosen metric
    display_delta_histogram(metric_name_input, delta_test1)
else:
    print("Invalid metric name. Please choose from the following metric names:")
    print(metric_names)

## Deltas distributions types using distfit package

In [1]:
from multiprocessing import Pool
from distribution_fit import fit_distribution
def fit_distributions(metric_names, test_data, control_data, test_data_name, verbose=False, num_processes=None):
    deltas = []
    for metric_name in metric_names:
        delta = calculate_delta(metric_name, test_data[metric_name], control_data[metric_name])
        delta = delta[delta>0]
        deltas.append((metric_name, delta))
    with Pool(num_processes) as pool:
        results = pool.starmap(fit_distribution, deltas)
    result_table = pd.DataFrame(results, columns=['Metric', 'Best Fitted Distribution', 'Score', 'Loc', 'Scale', 'Params'])
    return result_table

In [ ]:
result_table_delta = fit_distributions(metric_names, data1_test1, data1_control, "data1_test1", verbose=True, num_processes=6)
result_table_delta

# Corectness and sensitivity (A/A & A/B testing)

In [11]:
from multiprocessing import Pool
from salty import run_salt
def check_test(data, nominator, split_count, method, effects, denominator=None, N=10, split_by='userid', num_processes=None):
    
    p_values = []
    power = []

    df = data.copy()

    with Pool(num_processes) as pool:
        results = []

        for salt in tqdm(range(N)):
            results.append(pool.apply_async(run_salt, (salt, df, split_count, nominator, method, effects, denominator, split_by)))

        for result in tqdm(results):
            p_values_per_combo, power_per_combo = result.get()
            p_values.append(p_values_per_combo)
            power.append(power_per_combo)
    return np.array(p_values), np.array(power)

In [ ]:
effects = []
for i in [1,2,3,4,5,10,12.5,15]:
    effect = [0, i/100, i/100]
    effects.append(effect)

all_results = []

for metric_name in metric_names:
    metric_results = []
    for effect_set in effects:
        p_values, power = check_test(data1, metric_name, 2, scipy.stats.ttest_ind, effect_set, denominator=None, N=1000, split_by='test_user_id', num_processes=12)
        fpr1 = 100 * sum(np.array(p_values[:,0]) < 0.05) / len(p_values[:,0])
        sens1 = 100 * sum(np.array(power[:,0]) < 0.05) / len(power[:,0])
        effect_sizes = ';'.join([str(effect_size) for effect_size in effect_set])
        metric_results.append([f"{fpr1:.2f}%", f"{sens1:.2f}%"])

    all_results.append([metric_name] + metric_results)

table = pd.DataFrame(all_results, columns=['Metric'] + [f"({';'.join([str(effect_size) for effect_size in effect])})" for effect in effects])
table.set_index('Metric', inplace=True)

# MDE

In [36]:
import numpy as np

FIRST_TYPE_ERROR = 0.05
SECOND_TYPE_ERROR = 0.2
ddof = 1


def get_parameter_size(
    nominator,
    parameter,
    split_count=2,
    effect=0.01,
    denominator=None,
    ratio=1.0,
    alpha=FIRST_TYPE_ERROR,
    beta=SECOND_TYPE_ERROR,
    alternative="two-sided",
    ddof=1,
):
    if denominator is None:
        denominator = np.ones(len(nominator))

    pairing_cnt = split_count * (split_count - 1) / 2
    alpha = alpha / pairing_cnt
    
    mean, std = np.mean(nominator), np.std(nominator / denominator, ddof=ddof)
    nobs1 = len(nominator) / (1 + ratio)

    if parameter == "effect":
        return get_effect_size(mean, std, nobs1, ratio=ratio, alpha=alpha, beta=beta, alternative=alternative)
    elif parameter == "size":
        return get_sample_size(mean, std, effect, ratio=ratio, alpha=alpha, beta=beta, alternative=alternative)
    elif parameter == "power":
        return get_power_size(mean, std, nobs1, effect, ratio=ratio, alpha=alpha, alternative=alternative)
    elif parameter == "correctness":
        return get_correctness_size(mean, std, nobs1, effect, ratio=ratio, beta=beta, alternative=alternative)
    else:
        raise Exception('Uknown parameter. Use from available "effect", "size", "power", "correctness"')


def get_effect_size(
    mean, std, nobs1, alpha=FIRST_TYPE_ERROR, beta=SECOND_TYPE_ERROR, ratio=1.0, alternative="two-sided"
):
    std_effect = tt_ind_solve_power(
        effect_size=None, nobs1=nobs1, alpha=alpha, power=1 - beta, ratio=ratio, alternative=alternative
    )
    mde = std_effect * std / mean

    return np.round(100.0 * mde, 2)


def get_sample_size(
    mean, std, effect, alpha=FIRST_TYPE_ERROR, beta=SECOND_TYPE_ERROR, ratio=1.0, alternative="two-sided"
):
    std_effect = mean * effect / std

    sample_size = tt_ind_solve_power(
        effect_size=std_effect, nobs1=None, alpha=alpha, power=1 - beta, ratio=ratio, alternative=alternative
    )

    return int(sample_size * (1 + ratio))


def get_power_size(mean, std, nobs1, effect, alpha=FIRST_TYPE_ERROR, ratio=1.0, alternative="two-sided"):
    std_effect = mean * effect / std

    power_size = tt_ind_solve_power(
        effect_size=std_effect, nobs1=nobs1, alpha=alpha, power=None, ratio=ratio, alternative=alternative
    )

    return np.round(100.0 * power_size, 2)


def get_correctness_size(mean, std, nobs1, effect, beta=SECOND_TYPE_ERROR, ratio=1.0, alternative="two-sided"):
    std_effect = mean * effect / std

    correctness_size = tt_ind_solve_power(
        effect_size=std_effect, nobs1=nobs1, alpha=None, power=1 - beta, ratio=ratio, alternative=alternative
    )

    return np.round(100.0 * correctness_size, 2)

In [ ]:
def get_sample_sizes(data, metric_names, split_count=2, denominator=None, ratio=1.0, alpha=0.05, beta=0.2, alternative="two-sided", ddof=1):
    effect_sizes = [0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.125, 0.15, 0.20]
    sample_sizes = {}

    for effect in effect_sizes:
        sample_sizes[effect] = {}
        for metric_name in metric_names:
            metric_data = data1_control[metric_name]
            size = get_parameter_size(metric_data, "size", split_count, effect, denominator, ratio, alpha, beta, alternative, ddof)
            sample_sizes[effect][metric_name] = size

    return pd.DataFrame(sample_sizes)
get_sample_sizes(data1_control, metric_names)

In [ ]:
def get_sample_sizes(data, metric_names, split_count=2, denominator=None, ratio=1.0, alpha=0.05, beta=0.2, alternative="two-sided", ddof=1):
    effect_sizes = [0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.125, 0.15, 0.20]
    sample_sizes = {}
    for metric_name in metric_names:
        metric_data = data1_control[metric_name]
        effect1 = get_parameter_size(metric_data, "effect", split_count, None, denominator, ratio, alpha, beta, alternative, ddof)
        sample_sizes[metric_name] = [effect1]

    return pd.DataFrame(sample_sizes)
get_sample_sizes(data1_control, metric_names)

## CUPED

In [39]:
data4 = pd.read_csv('similars_new_model_v2.csv')
data3 = pd.read_csv('similars_new_model_v2_pred.csv')

In [40]:
data3 = data3[data3['has_completed_film']==1]
data4 = data4[data4['has_completed_film']==1]

In [41]:
data_cuped=pd.merge(data3, data4, on='test_user_id')
data_cuped['twt_similar_after_watch_x'] = data_cuped['twt_similar_after_watch_x']/3600
data_cuped['twt_similar_after_watch_y'] = data_cuped['twt_similar_after_watch_y']/3600

data_cuped['twt_x'] = data_cuped['twt_x']/3600
data_cuped['twt_y'] = data_cuped['twt_y']/3600

### Linear regression (Average Treatment Effect) before CUPED

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
np.mean(data_cuped.loc[data_cuped["tag_x"]=='test1', 'twt_similar_after_watch_y']) - np.mean(data_cuped.loc[data_cuped["tag_x"]=='control', 'twt_similar_after_watch_y'])
print(smf.ols('twt_similar_after_watch_y ~ tag_x', data=data_cuped).fit().summary().tables[1])

In [ ]:
theta = smf.ols('twt_similar_after_watch_y ~ twt_similar_after_watch_x', data=data_cuped).fit().params[1]
theta

In [ ]:
data_cuped['twt_similar_after_watch_cuped'] = data_cuped['twt_similar_after_watch_y'] - theta * (data_cuped['twt_similar_after_watch_x'] - np.mean(data_cuped['twt_similar_after_watch_x']))
data_cuped

### Linear regression (Average Treatment Effect) after CUPED

In [ ]:
print(smf.ols('twt_similar_after_watch_cuped ~ tag_x', data=data_cuped).fit().summary().tables[1])

In [ ]:
data_cuped["twt_similar_after_watch_cuped"].describe()

### Correctness and sensitivity for CUPED

In [ ]:
metric_names = ['twt_similar_after_watch_cuped', 'twt_similar_after_watch_y']
effects = []
for i in [1,2,3,4,5]:
    effect = [0, i/100]
    effects.append(effect)

all_results = []

for metric_name in metric_names:
    metric_results = []
    for effect_set in effects:
        p_values, power = check_test(data_cuped, metric_name, 2, scipy.stats.ttest_ind, effect_set, denominator=None, N=1000, split_by='test_user_id', num_processes=12)
        fpr1 = 100 * sum(np.array(p_values[:,0]) < 0.05) / len(p_values[:,0])
        sens1 = 100 * sum(np.array(power[:,0]) < 0.05) / len(power[:,0])
        effect_sizes = ';'.join([str(effect_size) for effect_size in effect_set])
        metric_results.append([f"{fpr1:.2f}%", f"{sens1:.2f}%"])

    all_results.append([metric_name] + metric_results)

table = pd.DataFrame(all_results, columns=['Metric'] + [f"({';'.join([str(effect_size) for effect_size in effect])})" for effect in effects])
table.set_index('Metric', inplace=True)

In [ ]:
#table.to_csv('cuped_v2.csv')
table